# How COVID-19 impacted U.S Automotive Accidents?
### Data Engineering Capstone Project
#### Preparing queries for Analytics

##### Reading Config File and Setting up AWS Redshift Connection string

In [ ]:
%load_ext sql

In [ ]:
import configparser

# Define config_file
config_file = 'dwh.cfg'

# Read CFG File
config = configparser.ConfigParser()
config.read(config_file)

# Variables to create connection to Redshift Cluster
host = config.get('CLUSTER', 'HOST')
db_name = config.get('CLUSTER', 'DB_NAME')
db_username = config.get('CLUSTER', 'DB_USER')
db_password = config.get('CLUSTER', 'DB_PASSWORD')
port = config.getint('CLUSTER', 'DB_PORT')

# Connecting to Redshift Cluster
conn_string = "postgresql://{}:{}@{}:{}/{}".format(db_username, db_password, host, port, db_name)

%sql $conn_string

## Entity Relationship Diagram

![data_model](../images/data-model.png)

## Total Number of Accidents in 2019 Breakdown Month

In [ ]:
%%sql 
select b.month_number
      ,b.month_name as month
      ,count(1) as total_accidents
from fact.us_accidents as a
inner join dim.dates as b on a.date_id = b.date_id
where b.year_number = 2019
group by b.month_number, b.month_name
order by 1

## Total Number of Accidents in 2020 Where COVID-19 Cases Were Confirmed Breakdown Month

In [ ]:
%%sql 
select b.month_number
      ,b.month_name as month
      ,count(1) as total_accidents
from fact.us_accidents as a
inner join dim.dates as b on a.date_id = b.date_id
inner join dim.accident_address as c on a.accident_address_id = c.accident_address_id
inner join fact.us_covid_19 as d on a.date_id = d.date_id and c.state = d.state and c.county = d.county
where b.year_number = 2020 and (d.confirmed > 0 or d.deaths > 0)
group by b.month_number, b.month_name
order by 1

## Total Number of Accidents Breakdown Weather Condition

In [ ]:
%%sql 
select b.weather_condition
      ,count(1) as total_accidents
from fact.us_accidents as a
inner join dim.weather_condition as b on a.weather_condition_id = b.weather_condition_id
group by b.weather_condition
order by 2 desc

## Total Number of Accidents Breakdown Hour

In [ ]:
%%sql 
select extract(HOUR FROM start_time) As hour
      ,count(1) as total_accidents
from fact.us_accidents
Group by extract(HOUR FROM start_time)
Order By 1 Asc